In [2]:
import pandas as pd
import polars as pl
import numpy as np
import os
from tqdm.auto import tqdm
from matplotlib import pyplot as plt
import pickle

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.metrics import r2_score
from lightgbm import LGBMRegressor
import lightgbm as lgb
from xgboost import XGBRegressor
from sklearn.ensemble import VotingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR

import warnings
warnings.filterwarnings('ignore')
pd.options.display.max_columns = None

# import kaggle_evaluation.jane_street_inference_server

/Users/lihengpanza/miniforge3/envs/js_kaggle_venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
train = pl.scan_parquet("./preprocessed_dataset/training.parquet").collect().to_pandas()
valid = pl.scan_parquet("./preprocessed_dataset/validation.parquet").collect().to_pandas()
train.shape, valid.shape

((27256944, 129), (7397456, 129))

In [4]:
train.head()

,date_id,time_id,symbol_id,weight,feature_00,feature_01,feature_02,feature_03,feature_04,feature_05,feature_06,feature_07,feature_08,feature_09,feature_10,feature_11,feature_12,feature_13,feature_14,feature_15,feature_16,feature_17,feature_18,feature_19,feature_20,feature_21,feature_22,feature_23,feature_24,feature_25,feature_26,feature_27,feature_28,feature_29,feature_30,feature_31,feature_32,feature_33,feature_34,feature_35,feature_36,feature_37,feature_38,feature_39,feature_40,feature_41,feature_42,feature_43,feature_44,feature_45,feature_46,feature_47,feature_48,feature_49,feature_50,feature_51,feature_52,feature_53,feature_54,feature_55,feature_56,feature_57,feature_58,feature_59,feature_60,feature_61,feature_62,feature_63,feature_64,feature_65,feature_66,feature_67,feature_68,feature_69,feature_70,feature_71,feature_72,feature_73,feature_74,feature_75,feature_76,feature_77,feature_78,responder_0,responder_1,responder_2,responder_3,responder_4,responder_5,responder_6,responder_7,responder_8,partition_id,responder_0_lag_1_min,responder_0_lag_1_max,responder_0_lag_1_mean,responder_0_lag_1_sum,responder_1_lag_1_min,responder_1_lag_1_max,responder_1_lag_1_mean,responder_1_lag_1_sum,responder_2_lag_1_min,responder_2_lag_1_max,responder_2_lag_1_mean,responder_2_lag_1_sum,responder_3_lag_1_min,responder_3_lag_1_max,responder_3_lag_1_mean,responder_3_lag_1_sum,responder_4_lag_1_min,responder_4_lag_1_max,responder_4_lag_1_mean,responder_4_lag_1_sum,responder_5_lag_1_min,responder_5_lag_1_max,responder_5_lag_1_mean,responder_5_lag_1_sum,responder_6_lag_1_min,responder_6_lag_1_max,responder_6_lag_1_mean,responder_6_lag_1_sum,responder_7_lag_1_min,responder_7_lag_1_max,responder_7_lag_1_mean,responder_7_lag_1_sum,responder_8_lag_1_min,responder_8_lag_1_max,responder_8_lag_1_mean,responder_8_lag_1_sum
0,1000,0,0,3.324375,-0.276179,-0.655325,-0.404810,-0.349785,-2.882722,0.222446,2.041360,0.077636,-0.212900,11,7,76,-0.676669,0.433896,-0.355392,-0.325359,-0.492663,-0.312569,-2.078509,-1.272981,0.411098,-0.144264,0.866810,0.739595,0.312714,-0.260926,1.022410,2.206249,0.694807,-0.205259,-0.363403,-0.219138,-0.757111,-0.329624,-1.524193,-1.683511,-1.642109,-0.073022,-0.417487,0.535187,1.487399,0.721523,0.569957,-0.293067,0.759278,-1.505769,1.269841,1.114140,0.245846,0.316328,1.741328,2.114579,1.746721,0.676324,-0.946189,-0.114278,-0.960180,1.436662,0.913091,2.705352,1.023804,1.415883,-0.281040,-0.141356,-0.280767,-1.607651,-1.580315,-0.779298,0.226700,-0.343499,-1.106544,0.829147,-0.559327,-0.398754,-0.345258,-0.201428,-0.238426,-0.317972,-0.307707,-0.322415,0.021479,0.298220,0.207253,-0.629241,0.283405,0.496563,-0.696224,0.288554,5,-5.0,5.0,-0.004038,-136.791397,-2.510265,5.0,-0.029304,-992.809082,-5.0,5.0,0.000356,12.054842,-5.0,5.0,-0.005787,-196.050095,-4.482391,4.49442,-0.010955,-371.140717,-5.0,5.0,-0.006098,-206.614258,-5.0,5.0,0.000154,5.210876,-4.959575,5.0,0.016449,557.28479,-5.0,5.0,-0.004793,-162.399887
1,1000,0,1,4.711303,-0.418316,-0.762019,-0.433680,-0.616798,-2.577970,0.205137,2.825696,0.057949,-0.334825,11,7,76,-1.086459,0.187593,-0.468343,-0.661830,-0.715271,-0.533931,-2.100300,-1.407540,0.307116,0.023577,2.056575,1.128782,0.405507,-0.028294,0.411470,1.283333,1.513040,-0.577109,-0.717683,0.023880,-0.941301,0.112354,-0.443716,-0.916682,0.187367,-0.104276,-0.280721,1.313757,0.678723,0.894007,-0.140150,-0.552522,-0.254354,-1.539251,1.249265,0.383563,0.544622,0.424014,1.022142,1.396059,-0.799310,0.306575,1.002958,-0.186624,-1.783038,2.750445,1.651704,1.351758,0.904945,1.415883,-0.385043,-0.392026,-0.400931,-2.225126,-1.699629,-0.378554,0.140990,-0.509071,-0.801501,0.154954,-0.657456,-0.252326,-0.273913,-0.286799,-0.230067,-0.317660,-0.292564,-0.485403,-0.368653,-0.314810,0.136140,-0.228491,0.113280,0.529877,-0.063614,0.379624,5,-5.0,5.0,-0.004038,-136.791397,-2.510265,5.0,-0.029304,-992.809082,-5.0,5.0,0.000356,12.054842,-5.0,5.0,-0.005787,-196.050095,-4.482391,4.49442,-0.010955,-371.140717,-5.0,5.0,-0.006098,-206.614258,-5.0

In [5]:
valid.head()

,date_id,time_id,symbol_id,weight,feature_00,feature_01,feature_02,feature_03,feature_04,feature_05,feature_06,feature_07,feature_08,feature_09,feature_10,feature_11,feature_12,feature_13,feature_14,feature_15,feature_16,feature_17,feature_18,feature_19,feature_20,feature_21,feature_22,feature_23,feature_24,feature_25,feature_26,feature_27,feature_28,feature_29,feature_30,feature_31,feature_32,feature_33,feature_34,feature_35,feature_36,feature_37,feature_38,feature_39,feature_40,feature_41,feature_42,feature_43,feature_44,feature_45,feature_46,feature_47,feature_48,feature_49,feature_50,feature_51,feature_52,feature_53,feature_54,feature_55,feature_56,feature_57,feature_58,feature_59,feature_60,feature_61,feature_62,feature_63,feature_64,feature_65,feature_66,feature_67,feature_68,feature_69,feature_70,feature_71,feature_72,feature_73,feature_74,feature_75,feature_76,feature_77,feature_78,responder_0,responder_1,responder_2,responder_3,responder_4,responder_5,responder_6,responder_7,responder_8,partition_id,responder_0_lag_1_min,responder_0_lag_1_max,responder_0_lag_1_mean,responder_0_lag_1_sum,responder_1_lag_1_min,responder_1_lag_1_max,responder_1_lag_1_mean,responder_1_lag_1_sum,responder_2_lag_1_min,responder_2_lag_1_max,responder_2_lag_1_mean,responder_2_lag_1_sum,responder_3_lag_1_min,responder_3_lag_1_max,responder_3_lag_1_mean,responder_3_lag_1_sum,responder_4_lag_1_min,responder_4_lag_1_max,responder_4_lag_1_mean,responder_4_lag_1_sum,responder_5_lag_1_min,responder_5_lag_1_max,responder_5_lag_1_mean,responder_5_lag_1_sum,responder_6_lag_1_min,responder_6_lag_1_max,responder_6_lag_1_mean,responder_6_lag_1_sum,responder_7_lag_1_min,responder_7_lag_1_max,responder_7_lag_1_mean,responder_7_lag_1_sum,responder_8_lag_1_min,responder_8_lag_1_max,responder_8_lag_1_mean,responder_8_lag_1_sum
0,1500,0,0,4.426604,0.299048,-2.586622,0.300780,0.685828,2.150813,-0.692489,-0.159566,-0.829518,0.505951,11,7,76,-0.973271,2.296871,-0.123932,-0.928972,-0.526217,-0.808886,-1.656028,-1.476867,0.454376,-0.049191,0.897194,0.927829,3.011863,0.280865,0.985659,0.529341,0.195879,-0.908160,-0.999317,-0.057682,-2.967964,-0.906526,-2.636707,-3.170421,-2.922452,-0.019737,0.283299,-1.159388,1.130700,-0.701951,-1.590536,-0.189725,-1.657003,-1.188985,1.142485,-0.677676,2.368523,-0.088314,-2.308198,2.195963,-1.299398,-0.930982,0.449019,-1.569682,-0.548544,2.261573,-1.144345,1.746195,0.641324,-0.644595,-0.527839,-0.293633,-0.495875,-1.291651,-1.418662,-0.738298,0.778600,-0.316974,-0.652625,2.652362,0.134199,0.275032,0.278624,2.722010,2.666984,0.572996,0.603494,-0.433041,0.073844,-0.385438,-1.170643,0.676503,-0.609011,-1.314976,0.819128,-0.747843,8,-5.0,5.0,-0.059639,-2251.505127,-5.0,5.0,-0.159751,-6030.923828,-5.0,5.0,-0.013975,-527.591187,-5.0,5.0,0.004935,186.320587,-5.0,5.0,0.034419,1299.403076,-5.0,5.0,-0.003542,-133.705368,-5.0,5.0,0.026963,1017.908264,-4.250791,5.0,0.089488,3378.341553,-5.0,5.0,0.003796,143.312149
1,1500,0,1,2.180898,0.084839,-2.740633,0.845043,-0.109019,2.475105,-1.215764,-0.196480,-0.740933,0.494199,11,7,76,-0.865345,2.347360,-0.008078,-0.625953,-0.766337,-0.594560,-1.044726,-2.061352,0.136830,0.122308,-0.102373,0.570324,2.777650,-0.858358,-1.851242,-0.743143,0.322359,-0.633344,-1.032915,0.108102,1.324808,-0.626489,0.624657,0.875317,-1.312669,0.109273,0.214136,-0.693444,0.124902,0.122866,-1.233770,-1.086170,-0.704810,-1.753582,1.150372,-0.293006,-0.907240,-0.437529,-1.296799,0.017573,-0.470545,-2.055546,-0.345634,-0.556286,-1.906425,1.674549,-0.967334,-0.706519,-0.469597,-0.644595,-0.339356,-0.346034,-0.331003,-1.533621,-1.740196,-0.552159,1.703114,0.008995,-1.000061,2.383818,-0.014400,0.282659,0.350185,-0.099467,-0.112053,-0.219843,-0.337088,0.434484,0.391614,-0.332026,0.097082,1.164387,-0.708231,-0.086879,1.134315,-1.159028,8,-5.0,5.0,-0.059639,-2251.505127,-5.0,5.0,-0.159751,-6030.923828,-5.0,5.0,-0.013975,-527.591187,-5.0,5.0,0.004935,186.320587,-5.0,5.0,0.034419,1299.403076,-5.0,5.0,-0.003542,-133.705368,-5.0,5.0,0.026963

In [6]:
class CONFIG:
    seed = 42
    target_col = "responder_6"
    cont_ftrs = ["weight"] + \
                [f"feature_{i:02d}" for i in range(9)] +  \
                [f"feature_{i:02d}" for i in range(12, 79)] + \
                [f"responder_{i}_lag_1_min" for i in range(9)] + \
                [f"responder_{i}_lag_1_max" for i in range(9)] + \
                [f"responder_{i}_lag_1_mean" for i in range(9)] + \
                [f"responder_{i}_lag_1_sum" for i in range(9)]
    onehot_ftrs = ["feature_09", "feature_10", "feature_11"]
    feature_cols = cont_ftrs + onehot_ftrs

In [7]:
X_train = train[ CONFIG.feature_cols ]
y_train = train[ CONFIG.target_col ]
w_train = train[ "weight" ]
X_valid = valid[ CONFIG.feature_cols ]
y_valid = valid[ CONFIG.target_col ]
w_valid = valid[ "weight" ]

X_train.shape, y_train.shape, w_train.shape, X_valid.shape, y_valid.shape, w_valid.shape

((27256944, 116),
 (27256944,),
 (27256944,),
 (7397456, 116),
 (7397456,),
 (7397456,))

In [8]:
# collect all the encoders into one preprocessor
preprocessor = ColumnTransformer(
    transformers=[
        ('onehot', OneHotEncoder(handle_unknown='ignore'), CONFIG.onehot_ftrs),
        ('std', StandardScaler(), CONFIG.cont_ftrs)])

prep = Pipeline(steps=[('preprocessor', preprocessor)])

In [9]:
X_train.head()

,weight,feature_00,feature_01,feature_02,feature_03,feature_04,feature_05,feature_06,feature_07,feature_08,feature_12,feature_13,feature_14,feature_15,feature_16,feature_17,feature_18,feature_19,feature_20,feature_21,feature_22,feature_23,feature_24,feature_25,feature_26,feature_27,feature_28,feature_29,feature_30,feature_31,feature_32,feature_33,feature_34,feature_35,feature_36,feature_37,feature_38,feature_39,feature_40,feature_41,feature_42,feature_43,feature_44,feature_45,feature_46,feature_47,feature_48,feature_49,feature_50,feature_51,feature_52,feature_53,feature_54,feature_55,feature_56,feature_57,feature_58,feature_59,feature_60,feature_61,feature_62,feature_63,feature_64,feature_65,feature_66,feature_67,feature_68,feature_69,feature_70,feature_71,feature_72,feature_73,feature_74,feature_75,feature_76,feature_77,feature_78,responder_0_lag_1_min,responder_1_lag_1_min,responder_2_lag_1_min,responder_3_lag_1_min,responder_4_lag_1_min,responder_5_lag_1_min,responder_6_lag_1_min,responder_7_lag_1_min,responder_8_lag_1_min,responder_0_lag_1_max,responder_1_lag_1_max,responder_2_lag_1_max,responder_3_lag_1_max,responder_4_lag_1_max,responder_5_lag_1_max,responder_6_lag_1_max,responder_7_lag_1_max,responder_8_lag_1_max,responder_0_lag_1_mean,responder_1_lag_1_mean,responder_2_lag_1_mean,responder_3_lag_1_mean,responder_4_lag_1_mean,responder_5_lag_1_mean,responder_6_lag_1_mean,responder_7_lag_1_mean,responder_8_lag_1_mean,responder_0_lag_1_sum,responder_1_lag_1_sum,responder_2_lag_1_sum,responder_3_lag_1_sum,responder_4_lag_1_sum,responder_5_lag_1_sum,responder_6_lag_1_sum,responder_7_lag_1_sum,responder_8_lag_1_sum,feature_09,feature_10,feature_11
0,3.324375,-0.276179,-0.655325,-0.404810,-0.349785,-2.882722,0.222446,2.041360,0.077636,-0.212900,-0.676669,0.433896,-0.355392,-0.325359,-0.492663,-0.312569,-2.078509,-1.272981,0.411098,-0.144264,0.866810,0.739595,0.312714,-0.260926,1.022410,2.206249,0.694807,-0.205259,-0.363403,-0.219138,-0.757111,-0.329624,-1.524193,-1.683511,-1.642109,-0.073022,-0.417487,0.535187,1.487399,0.721523,0.569957,-0.293067,0.759278,-1.505769,1.269841,1.114140,0.245846,0.316328,1.741328,2.114579,1.746721,0.676324,-0.946189,-0.114278,-0.960180,1.436662,0.913091,2.705352,1.023804,1.415883,-0.281040,-0.141356,-0.280767,-1.607651,-1.580315,-0.779298,0.226700,-0.343499,-1.106544,0.829147,-0.559327,-0.398754,-0.345258,-0.201428,-0.238426,-0.317972,-0.307707,-5.0,-2.510265,-5.0,-5.0,-4.482391,-5.0,-5.0,-4.959575,-5.0,5.0,5.0,5.0,5.0,4.49442,5.0,5.0,5.0,5.0,-0.004038,-0.029304,0.000356,-0.005787,-0.010955,-0.006098,0.000154,0.016449,-0.004793,-136.791397,-992.809082,12.054842,-196.050095,-371.140717,-206.614258,5.210876,557.28479,-162.399887,11,7,76
1,4.711303,-0.418316,-0.762019,-0.433680,-0.616798,-2.577970,0.205137,2.825696,0.057949,-0.334825,-1.086459,0.187593,-0.468343,-0.661830,-0.715271,-0.533931,-2.100300,-1.407540,0.307116,0.023577,2.056575,1.128782,0.405507,-0.028294,0.411470,1.283333,1.513040,-0.577109,-0.717683,0.023880,-0.941301,0.112354,-0.443716,-0.916682,0.187367,-0.104276,-0.280721,1.313757,0.678723,0.894007,-0.140150,-0.552522,-0.254354,-1.539251,1.249265,0.383563,0.544622,0.424014,1.022142,1.396059,-0.799310,0.306575,1.002958,-0.186624,-1.783038,2.750445,1.651704,1.351758,0.904945,1.415883,-0.385043,-0.392026,-0.400931,-2.225126,-1.699629,-0.378554,0.140990,-0.509071,-0.801501,0.154954,-0.657456,-0.252326,-0.273913,-0.286799,-0.230067,-0.317660,-0.292564,-5.0,-2.510265,-5.0,-5.0,-4.482391,-5.0,-5.0,-4.959575,-5.0,5.0,5.0,5.0,5.0,4.49442,5.0,5.0,5.0,5.0,-0.004038,-0.029304,0.000356,-0.005787,-0.010955,-0.006098,0.000154,0.016449,-0.004793,-136.791397,-992.809082,12.054842,-196.050095,-371.140717,-206.614258,5.210876,557.28479,-162.399887,11,7,76
2,3.028847,-0.724897,-1.223187,-0.452174,-0.523907,-2.617430,0.277329,3.187561,0.106466,-0.605264,-0.843919,1.438550,-0.277886,-0.560161,-0.230204,-0.607569,-1.796766,-2.026716,-1.291414,-0.150410,0.849451,0.358889,-0.965452,-1.009622,0.523362,1.558008

In [10]:
# RANDOM SAMPLE 1000000 ROWS TO FORM OUR TRAIN, VAL DF
X_train = X_train.sample(100000, random_state=CONFIG.seed)
y_train = y_train[X_train.index]
w_train = w_train[X_train.index]

X_valid = X_valid.sample(20000, random_state=CONFIG.seed)
y_valid = y_valid[X_valid.index]
w_valid = w_valid[X_valid.index]


df_train = prep.fit_transform(X_train)
feature_names = prep.get_feature_names_out()
df_train = pd.DataFrame(data=df_train, columns=feature_names, index=X_train.index)

df_valid = prep.transform(X_valid)
df_valid = pd.DataFrame(data=df_valid, columns=feature_names, index=X_valid.index)



In [11]:
df_train.head()

,onehot__feature_09_2,onehot__feature_09_4,onehot__feature_09_9,onehot__feature_09_11,onehot__feature_09_12,onehot__feature_09_14,onehot__feature_09_15,onehot__feature_09_26,onehot__feature_09_30,onehot__feature_09_34,onehot__feature_09_42,onehot__feature_09_44,onehot__feature_09_46,onehot__feature_09_50,onehot__feature_09_57,onehot__feature_09_64,onehot__feature_09_68,onehot__feature_09_70,onehot__feature_09_81,onehot__feature_09_82,onehot__feature_10_1,onehot__feature_10_2,onehot__feature_10_3,onehot__feature_10_4,onehot__feature_10_5,onehot__feature_10_6,onehot__feature_10_7,onehot__feature_10_10,onehot__feature_10_12,onehot__feature_11_9,onehot__feature_11_11,onehot__feature_11_13,onehot__feature_11_16,onehot__feature_11_24,onehot__feature_11_25,onehot__feature_11_34,onehot__feature_11_40,onehot__feature_11_48,onehot__feature_11_50,onehot__feature_11_59,onehot__feature_11_62,onehot__feature_11_63,onehot__feature_11_66,onehot__feature_11_76,onehot__feature_11_150,onehot__feature_11_158,onehot__feature_11_171,onehot__feature_11_214,onehot__feature_11_230,onehot__feature_11_261,onehot__feature_11_336,onehot__feature_11_376,onehot__feature_11_388,onehot__feature_11_410,onehot__feature_11_522,onehot__feature_11_534,onehot__feature_11_539,std__weight,std__feature_00,std__feature_01,std__feature_02,std__feature_03,std__feature_04,std__feature_05,std__feature_06,std__feature_07,std__feature_08,std__feature_12,std__feature_13,std__feature_14,std__feature_15,std__feature_16,std__feature_17,std__feature_18,std__feature_19,std__feature_20,std__feature_21,std__feature_22,std__feature_23,std__feature_24,std__feature_25,std__feature_26,std__feature_27,std__feature_28,std__feature_29,std__feature_30,std__feature_31,std__feature_32,std__feature_33,std__feature_34,std__feature_35,std__feature_36,std__feature_37,std__feature_38,std__feature_39,std__feature_40,std__feature_41,std__feature_42,std__feature_43,std__feature_44,std__feature_45,std__feature_46,std__feature_47,std__feature_48,std__feature_49,std__feature_50,std__feature_51,std__feature_52,std__feature_53,std__feature_54,std__feature_55,std__feature_56,std__feature_57,std__feature_58,std__feature_59,std__feature_60,std__feature_61,std__feature_62,std__feature_63,std__feature_64,std__feature_65,std__feature_66,std__feature_67,std__feature_68,std__feature_69,std__feature_70,std__feature_71,std__feature_72,std__feature_73,std__feature_74,std__feature_75,std__feature_76,std__feature_77,std__feature_78,std__responder_0_lag_1_min,std__responder_1_lag_1_min,std__responder_2_lag_1_min,std__responder_3_lag_1_min,std__responder_4_lag_1_min,std__responder_5_lag_1_min,std__responder_6_lag_1_min,std__responder_7_lag_1_min,std__responder_8_lag_1_min,std__responder_0_lag_1_max,std__responder_1_lag_1_max,std__responder_2_lag_1_max,std__responder_3_lag_1_max,std__responder_4_lag_1_max,std__responder_5_lag_1_max,std__responder_6_lag_1_max,std__responder_7_lag_1_max,std__responder_8_lag_1_max,std__responder_0_lag_1_mean,std__responder_1_lag_1_mean,std__responder_2_lag_1_mean,std__responder_3_lag_1_mean,std__responder_4_lag_1_mean,std__responder_5_lag_1_mean,std__responder_6_lag_1_mean,std__responder_7_lag_1_mean,std__responder_8_lag_1_mean,std__responder_0_lag_1_sum,std__responder_1_lag_1_sum,std__responder_2_lag_1_sum,std__responder_3_lag_1_sum,std__responder_4_lag_1_sum,std__responder_5_lag_1_sum,std__responder_6_lag_1_sum,std__responder_7_lag_1_sum,std__responder_8_lag_1_sum
2145488,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.535650,-0.631741,-0.766635,-0.954914,-0.948088,0.278891,-0.047051,-0.108702,0.033896,0.405188,-0.221769,-0.487554,-0.453303,-0.648023,-0.745711,-0.405036,0.499802,-1.090882,2.229815,0.039898,0.319465,-0.574945,1.493020,1.216924,-1.416103,-0.733892,0.324137,-0.468706,-0.348032,0.000699,-0.612218

In [12]:
df_valid.head()

,onehot__feature_09_2,onehot__feature_09_4,onehot__feature_09_9,onehot__feature_09_11,onehot__feature_09_12,onehot__feature_09_14,onehot__feature_09_15,onehot__feature_09_26,onehot__feature_09_30,onehot__feature_09_34,onehot__feature_09_42,onehot__feature_09_44,onehot__feature_09_46,onehot__feature_09_50,onehot__feature_09_57,onehot__feature_09_64,onehot__feature_09_68,onehot__feature_09_70,onehot__feature_09_81,onehot__feature_09_82,onehot__feature_10_1,onehot__feature_10_2,onehot__feature_10_3,onehot__feature_10_4,onehot__feature_10_5,onehot__feature_10_6,onehot__feature_10_7,onehot__feature_10_10,onehot__feature_10_12,onehot__feature_11_9,onehot__feature_11_11,onehot__feature_11_13,onehot__feature_11_16,onehot__feature_11_24,onehot__feature_11_25,onehot__feature_11_34,onehot__feature_11_40,onehot__feature_11_48,onehot__feature_11_50,onehot__feature_11_59,onehot__feature_11_62,onehot__feature_11_63,onehot__feature_11_66,onehot__feature_11_76,onehot__feature_11_150,onehot__feature_11_158,onehot__feature_11_171,onehot__feature_11_214,onehot__feature_11_230,onehot__feature_11_261,onehot__feature_11_336,onehot__feature_11_376,onehot__feature_11_388,onehot__feature_11_410,onehot__feature_11_522,onehot__feature_11_534,onehot__feature_11_539,std__weight,std__feature_00,std__feature_01,std__feature_02,std__feature_03,std__feature_04,std__feature_05,std__feature_06,std__feature_07,std__feature_08,std__feature_12,std__feature_13,std__feature_14,std__feature_15,std__feature_16,std__feature_17,std__feature_18,std__feature_19,std__feature_20,std__feature_21,std__feature_22,std__feature_23,std__feature_24,std__feature_25,std__feature_26,std__feature_27,std__feature_28,std__feature_29,std__feature_30,std__feature_31,std__feature_32,std__feature_33,std__feature_34,std__feature_35,std__feature_36,std__feature_37,std__feature_38,std__feature_39,std__feature_40,std__feature_41,std__feature_42,std__feature_43,std__feature_44,std__feature_45,std__feature_46,std__feature_47,std__feature_48,std__feature_49,std__feature_50,std__feature_51,std__feature_52,std__feature_53,std__feature_54,std__feature_55,std__feature_56,std__feature_57,std__feature_58,std__feature_59,std__feature_60,std__feature_61,std__feature_62,std__feature_63,std__feature_64,std__feature_65,std__feature_66,std__feature_67,std__feature_68,std__feature_69,std__feature_70,std__feature_71,std__feature_72,std__feature_73,std__feature_74,std__feature_75,std__feature_76,std__feature_77,std__feature_78,std__responder_0_lag_1_min,std__responder_1_lag_1_min,std__responder_2_lag_1_min,std__responder_3_lag_1_min,std__responder_4_lag_1_min,std__responder_5_lag_1_min,std__responder_6_lag_1_min,std__responder_7_lag_1_min,std__responder_8_lag_1_min,std__responder_0_lag_1_max,std__responder_1_lag_1_max,std__responder_2_lag_1_max,std__responder_3_lag_1_max,std__responder_4_lag_1_max,std__responder_5_lag_1_max,std__responder_6_lag_1_max,std__responder_7_lag_1_max,std__responder_8_lag_1_max,std__responder_0_lag_1_mean,std__responder_1_lag_1_mean,std__responder_2_lag_1_mean,std__responder_3_lag_1_mean,std__responder_4_lag_1_mean,std__responder_5_lag_1_mean,std__responder_6_lag_1_mean,std__responder_7_lag_1_mean,std__responder_8_lag_1_mean,std__responder_0_lag_1_sum,std__responder_1_lag_1_sum,std__responder_2_lag_1_sum,std__responder_3_lag_1_sum,std__responder_4_lag_1_sum,std__responder_5_lag_1_sum,std__responder_6_lag_1_sum,std__responder_7_lag_1_sum,std__responder_8_lag_1_sum
2999095,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.814319,1.257527,-0.218510,1.276750,1.936195,-1.004007,-0.290064,-0.123782,-0.328785,0.690101,-0.171653,-0.079009,-0.267276,-0.725592,-0.648014,-0.775463,0.184557,-1.368198,-1.320360,-0.244633,-1.009617,-0.687541,0.829493,1.197318,-0.325823,-0.922633,-0.609217,-0.395812,-0.840852,-0.346913,0.015

In [13]:
df_train.shape, df_valid.shape

((100000, 170), (20000, 170))

In [14]:
# Support Vector Machine Model
def SVM_model(seed):
    # SVM parameters
    SVM_Params = {
        'gamma': 1e-7,
        'C': 1e5,
        'kernel': 'rbf',
    }
    
    SVM_Model = SVR(**SVM_Params)
    return SVM_Model

# Random Forest Model
def RF_model(seed):
    # Random Forest parameters
    RF_Params = {
        'n_estimators': 100,
        'max_depth': 20,
        'random_state': seed
    }
    
    RF_Model = RandomForestRegressor(**RF_Params)
    return RF_Model


# XGBoost Model
def XGB_model(seed):
    # XGBoost parameters
    XGB_Params = {
        'learning_rate': 0.05,
        'max_depth': 6,
        'n_estimators': 200,
        'subsample': 0.8,
        'colsample_bytree': 0.8,
        'reg_alpha': 1,
        'reg_lambda': 5,
        'random_state': seed,
        'tree_method': 'hist',
    }
    
    XGB_Model = XGBRegressor(**XGB_Params)
    return XGB_Model

In [15]:
%%time
model = RF_model(CONFIG.seed)
model.fit(df_train, y_train)

CPU times: user 17min 11s, sys: 3.3 s, total: 17min 14s
Wall time: 17min 22s


RandomForestRegressor(max_depth=20, random_state=42)

In [16]:
y_pred_train = model.predict(df_train)
train_score = r2_score(y_train, y_pred_train, sample_weight=w_train )
train_score

0.11306091629333914

In [17]:
y_pred_valid = model.predict(df_valid)
valid_score = r2_score(y_valid, y_pred_valid, sample_weight=w_valid )
valid_score

0.0005463881061417153

In [18]:
import sklearn

In [19]:
result = {
    "model" : model,
    "preprocessor" : preprocessor,
}

with open("result.pkl", "wb") as fp:
    pickle.dump(result, fp)

In [20]:
# y_means = { symbol_id : -1 for symbol_id in range(39) }
# for symbol_id, gdf in train[["symbol_id", CONFIG.target_col]].groupby("symbol_id"):
#     y_mean = gdf[ CONFIG.target_col ].mean()
#     y_means[symbol_id] = y_mean
#     print(f"symbol_id = {symbol_id}, y_means = {y_mean:.5f}")

In [21]:
# cv_detail = { symbol_id : 0 for symbol_id in range(39) }
# for symbol_id, gdf in valid.groupby("symbol_id"):
#     X_valid = gdf[ CONFIG.feature_cols ]
#     y_valid = gdf[ CONFIG.target_col ]
#     w_valid = gdf[ "weight" ]
#     y_pred_valid = model.predict(X_valid)
#     score = r2_score(y_valid, y_pred_valid, sample_weight=w_valid )
#     cv_detail[symbol_id] = score
    
#     print(f"symbol_id = {symbol_id}, score = {score:.5f}")

In [22]:
# sids = list(cv_detail.keys())
# plt.bar(sids, [cv_detail[sid] for sid in sids])
# plt.grid()
# plt.xlabel("symbol_id")
# plt.ylabel("CV score")
# plt.show()